# Graph Embeddings

## Introduction
I recently learned that people are trying to use deep learning to solve problems from graph theory after a friend linked me the paper, [Learning Combinatorial Optimization Algorithms over Graphs](https://arxiv.org/pdf/1704.01665.pdf). I find this subject very interesting and want to explore it further. In this paper, I will try to create some simple graph representations by using vertex embeddings and a neural network. I am going to try to create what is essentially a lookup table using deep learning.

This is also a great exercise for me to learn how to use PyTorch


## Data
To create the graph, I will use a simple dictionary of nodes that contains a set of the neighbors. I will also define a next function so we can get batches of data to train the neural network.

In [1]:
from fastai.model import fit

In [2]:
from random import sample, randint, choice

In [3]:
import numpy as np

In [4]:
from torch import Tensor as T
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
class Graph:
    def __init__(self, n_vert = 10, n_neigh = 3, bs=64):
        """
        Initialize a graph object. Note that graph is not guaranteed to be
        fully connected
        
        TODO:
        Graph connectivity visualization
        """
        self.bs = bs
        self.vertices = range(n_vert)
        self.graph = {}
        for v in self.vertices:
            possible = set(self.vertices).difference({v}) # remove loops
            edges = set(sample(possible, n_neigh)) # add n neighbors to each vertex
            self.graph[v] = edges
            
    def pos_sample(self):
        v = choice(self.vertices)
        x = [v, choice(tuple(self.graph[v]))]
        return x
    
    def neg_sample(self):
        v = choice(self.vertices)
        possible = set(self.vertices).difference(self.graph[v])
        possible = possible.difference({v})
        x = [v, choice(tuple(possible))]
        return x
    
    def __iter__(self, bs=64):
        """
        Set the batch size
        
        TODO: put this in init
        """
        self.bs = bs
        return self
        
    def __next__(self):
        """
        Returns a batch of data for use in training. The vertices are randomly chosen
        so it is not guaranteed that all vertices are in each training step - or that
        every possible neighbor will be chosen. Positive and negative examples are 
        one hot encoded for simplicity
        """
        n_pos = int(self.bs / 2)
        n_neg = self.bs - n_pos
        
        pos = [[self.pos_sample(), [1, 0]] for _ in range(n_pos)]
        neg = [[self.neg_sample(), [0, 1]] for _ in range(n_neg)]
        X, Y = zip(*(pos + neg))
        X = list(zip(*X))
        return X, Y

In [6]:
n_vert = 10
n_neigh = 3
bs = 64
graph = Graph(n_vert=n_vert, n_neigh=n_neigh, bs=bs)

In [7]:
graph.graph

{0: {4, 7, 8},
 1: {2, 7, 9},
 2: {4, 6, 9},
 3: {1, 6, 7},
 4: {0, 8, 9},
 5: {0, 2, 7},
 6: {0, 2, 8},
 7: {0, 1, 6},
 8: {1, 3, 5},
 9: {0, 1, 5}}

In [8]:
X, Y = next(graph)

In [9]:
# %timeit next(graph) # data is generated at 5,500 times per second. speed should not be an issue

In [10]:
class Model(nn.Module):
    def __init__(self, n_vert, n_neigh, nh=10, p1=0.2, p2=0.2):
        super().__init__()
        
        emb_dim = 5 # dimension of the vertex embedding
        
        self.v = nn.Embedding(n_vert, emb_dim)
        self.v.weight.data.uniform_(-1, 1)
        
        self.lin1 = nn.Linear(emb_dim*2, nh)
        self.lin2 = nn.Linear(nh, 2)
        self.drop1 = nn.Dropout(p1)
        self.drop2 = nn.Dropout(p2)
        
    def forward(self, vertices):
        start = vertices[0,:]
        end = vertices[1,:]
        x = self.drop1(torch.cat([model.v(start), model.v(end)], dim=1))
        x = self.drop2(F.relu(self.lin1(x)))
        x = F.softmax(self.lin2(x), dim=1)
        return x

In [11]:
X, Y = next(graph)
inp = Variable(torch.LongTensor(X))

In [21]:
model = Model(n_vert, n_neigh).cuda()

In [13]:
from torch import optim

In [26]:
wd=1e-6
opt = optim.Adam(model.parameters(), 1e-3, weight_decay=wd)
criterion = F.cross_entropy

In [15]:
model

Model(
  (v): Embedding(10, 5)
  (lin1): Linear(in_features=10, out_features=10, bias=True)
  (lin2): Linear(in_features=10, out_features=2, bias=True)
  (drop1): Dropout(p=0.2)
  (drop2): Dropout(p=0.2)
)

In [36]:
# get batch of data
X, Y = next(graph)
inputs = Variable(torch.LongTensor(X)).cuda()
labels = Variable(torch.LongTensor(Y)).cuda()

RuntimeError: Variable data has to be a tensor, but got tuple

In [33]:
# zero gradients
opt.zero_grad()

In [34]:
# forward
outputs = model(inputs)
_, preds = torch.max(outputs.data, 1)

In [35]:
criterion(preds, labels)

RuntimeError: log_softmax(): argument 'input' (position 1) must be Variable, not torch.cuda.LongTensor

In [39]:
torch.max(Variable(torch.LongTensor(Y)).cuda(), 1)

(Variable containing:
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
 [torch.cuda.LongTensor of size 64 (GPU 0)], Variable containing:
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
 [torch.cuda.LongTensor of size 64 (GPU 0)])